<a href="https://colab.research.google.com/github/Sanazzzmi/Sentiment_analysis_of_the_Persian_text_with_word_embedding_-_frequency_word/blob/main/Sentiment_analysis_of_the_Persian_text_with_word_embedding_%26_frequency_word_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install and import data

[download lexicon url]( https://github.com/Text-Mining/Persian-Sentiment-Resources/blob/master/PerSent.xlsx)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hazm
!test -f resources-0.5.zip || curl -LO https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
!test -d resources || ( mkdir -p resources && cd resources && unzip ../resources-0.5.zip )

import hazm
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import random

#dataset
!wget https://raw.githubusercontent.com/ashalogic/Persian-Sentiment-Analyzer/master/Tutorial_Dataset.csv
csv_dataset = pd.read_csv("/content/Tutorial_Dataset.csv")

#lexicon
################## Saved address of the downloaded lexicon file #####################

lexicon = pd.read_excel("/content/drive/MyDrive/PerSent.xlsx")

################### pd.read_excel("Saved address of the downloaded lexicon file") ###

     |████████████████████████████████| 316 kB 10.3 MB/s 
     |████████████████████████████████| 1.4 MB 37.3 MB/s 
     |████████████████████████████████| 233 kB 7.3 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=96b32b6504547810482b24cc430e12793b3c6ddcc2a2677f2f46a01f1050f544
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153943 sha256=26bebc6d02b3bccf109682a17a5a1357f78ee6c31e2d9bde0aab685f2e3538f3
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  U

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
!gunzip /content/cc.fa.300.bin.gz
!pip install fasttext

import fasttext 

%time
model = fasttext.load_model("/content/cc.fa.300.bin")

--2022-01-25 09:54:14--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘cc.fa.300.bin.gz’

cc.fa.300.bin.gz    100%[===================>]   4.19G  33.9MB/s    in 1m 55s  

2022-01-25 09:56:10 (37.5 MB/s) - ‘cc.fa.300.bin.gz’ saved [4502524724/4502524724]

     |████████████████████████████████| 68 kB 6.1 MB/s 
  Using cached pybind11-2.9.0-py2.py3-none-any.whl (210 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3127585 sha256=7f24c2ba7fb8e774c0127430faf4d6d6e71bb9cd0b7a7acb425e8a027aaf973c
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built 

### train test split for CNN modle

In [ ]:
def CleanPersianText(text):
  _normalizer = hazm.Normalizer()
  text = _normalizer.normalize(text)
  return text

revlist = list(map(lambda x: [CleanPersianText(x[0]),x[1]],zip(csv_dataset['Text'],csv_dataset['Suggestion'])))
pos=list(filter(lambda x: x[1] == 1,revlist))
nat=list(filter(lambda x: x[1] == 2,revlist))
neg=list(filter(lambda x: x[1] == 3,revlist))

revlist_shuffle = pos[:450] + neg[:450]
random.shuffle(revlist_shuffle)

print("Posetive count {}".format(len(pos)))
print("Negetive count {}".format(len(neg)))
print("Natural  count {}".format(len(nat)))
print()
print("Total    count {}".format(len(revlist)))
print()
print("Posetive count : ","\n",pos[random.randrange(1,len(pos))])
print("Negetive count : ","\n",neg[random.randrange(1,len(neg))])
print("unknown  count : ","\n",nat[random.randrange(1,len(nat))])
print("Total    count {}".format(len(revlist_shuffle)))

Posetive count 2382
Negetive count 460
Natural  count 419

Total    count 3261

Posetive count :  
 ['لطفا هر چه زود\u200cتر موجود بشه میخام بخرم خیلی ممنون ', 1]
Negetive count :  
 ['هزینه خرید کارتریج بعد از مدت کوتاهی از هزینه خود دستگاه بیشتر میشه کارتیج این دستگاه خیلی گرونه ', 3]
unknown  count :  
 ['من ۱۰ ماه هست گرفتم و ازش راضی بودم تا اینکه امروز در بین کار خاموش شد و دیگه روشن نشد ', 2]
Total    count 900


In [ ]:
vector_size = 300 
max_no_tokens = 20 
import numpy as np
import keras.backend as K
train_size = int(0.9*(len(revlist_shuffle)))
test_size = int(0.1*(len(revlist_shuffle)))#lstm
indexes = set(np.random.choice(len(revlist_shuffle), train_size + test_size, replace=False))

x_train = np.zeros((train_size, max_no_tokens, vector_size), dtype=K.floatx())
y_train = np.zeros((train_size, 2), dtype=np.int32)

x_test = np.zeros((test_size, max_no_tokens, vector_size), dtype=K.floatx())
y_test = np.zeros((test_size, 2), dtype=np.int32)


for i, index in enumerate(indexes):
  text_words = hazm.word_tokenize(revlist_shuffle[index][0])
  for t in range(0,len(text_words)):
    if t >= max_no_tokens:
      break

    if text_words[t] not in model.words:
      continue
    if i < train_size:
      x_train[i, t, :] = model.get_word_vector(text_words[t])
    else:
      x_test[i - train_size, t, :] = model.get_word_vector(text_words[t])

  if i < train_size:
    y_train[i, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
  else:
    y_test[i - train_size, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
   
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((810, 20, 300), (90, 20, 300), (810, 2), (90, 2))

### **Article idea**

In [ ]:
text = [x[0] for x in revlist_shuffle ]
y = [x[1] for x in revlist_shuffle ]

In [ ]:
from keras.preprocessing.text import Tokenizer
token = Tokenizer()
token.fit_on_texts(text)

In [ ]:
vocab_size  = len(token.word_index) + 1
vocab_size

In [ ]:
encoded_text = token.texts_to_sequences(text)
print(encoded_text[:3])

[[171, 14, 1426, 122, 70, 5, 159, 2, 30, 101, 26, 47, 61], [36, 17, 42, 93, 11, 3154, 1, 930, 15], [4, 14, 18, 27, 61, 413, 34, 25, 4, 79, 2160, 2, 106, 23, 28, 71, 164, 310, 8, 437, 7, 43, 103, 1, 104, 611, 94, 845, 1234, 11, 2161]]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')


In [ ]:
import numpy as np
d = 300
word_vector_matrix = np.zeros((vocab_size, d))


not_word = []
have_word = []

for word, index in token.word_index.items()  :# هر کلمه یه شناسه داره
    vector = model.get_word_vector(word)
    if vector is not None:
        word_vector_matrix[index] = vector
        have_word.append(word)
    else:
        not_word.append(word)    

In [ ]:
yy = []
for x in y:
  if x == 1:
    yy.append(1)
  else:
    yy.append(0)

In [ ]:
#Divide texts by labels one and zero
df_y = pd.DataFrame(np.array(yy),columns=['a'])
df_one = df_y[df_y['a']==1].index.values.astype(int)
df_zero = df_y[df_y['a']==0].index.values.astype(int)

text_one = []
for x in df_one:
    text_one.append(text[x])

text_zero = []
for x in df_zero:
    text_zero.append(text[x])

join_text_zero = [' '.join(text_zero)]
join_text_one = [' '.join(text_one)]

df_join_text_zero = pd.DataFrame(join_text_zero[0:],columns= ["tex"])
df_join_text_one = pd.DataFrame(join_text_one[0:],columns= ["tex"])

df_have_word = pd.DataFrame({'word':have_word})
df_token = pd.DataFrame.from_dict(token.word_index.items())

df_text_one = pd.DataFrame(text_one[0:],columns= ["tex"])
df_text_zero = pd.DataFrame(text_zero[0:],columns= ["tex"])

from collections import Counter
count_zero = df_join_text_zero['tex'].str.split().apply(Counter)
count_one = df_join_text_one['tex'].str.split().apply(Counter)

df_counter_zero = pd.DataFrame.from_dict(count_zero[0], orient='index').reset_index()
df_counter_one = pd.DataFrame.from_dict(count_one[0], orient='index').reset_index()
df_counter_zero.columns = ['word', 'value']
df_counter_one.columns = ['word', 'value']

df_token.columns = ['word', 'id']

sentiment_zero =df_have_word.merge(df_counter_zero, left_on='word', right_on='word')[['value','word']]
sentiment_one = df_have_word.merge(df_counter_one, left_on='word', right_on='word')[['value','word']]

#Words found in both positive and negative sentences
common = sentiment_one.merge(sentiment_zero,on=['word','word'])
common.columns = ['value_one', 'word' , 'value_zero']

common["max"] = common[["value_one", "value_zero"]].max(axis=1)
common["min"] = common[["value_one", "value_zero"]].min(axis=1)


common["lable"] = np.where(common["value_one"] == common["max"], 1, 0)
common["frequent_negative_sentence"] = np.where(common["lable"] == 1 , common["min"], common["max"])
common["frequent_positive_sentence"] = np.where(common["lable"] == 0 , common["min"], common["max"])

common_df_final = common.drop('value_one', 1)
common_df_final = common_df_final.drop('value_zero', 1)
common_df_final = common_df_final.drop('max', 1)
common_df_final = common_df_final.drop('min', 1)

#Those words that are seen in positive sentences and not seen in negative sentences
just_one = sentiment_one[(~sentiment_one.word.isin(common.word))&(~sentiment_one.word.isin(common.word))]
just_one = just_one.assign(lable = 1)
just_one = just_one.assign(frequent_negative_sentence = 0)
just_one.columns = ['frequent_positive_sentence', 'word' , 'lable','frequent_negative_sentence']

#Those words that are seen in negative sentences and not seen in positive sentences
just_zero = sentiment_zero[(~sentiment_zero.word.isin(common.word))&(~sentiment_zero.word.isin(common.word))]
just_zero = just_zero.assign(lable = 0)
just_zero = just_zero.assign(frequent_posoitve_sentence = 0)
just_zero.columns = ['frequent_negative_sentence', 'word' , 'lable','frequent_positive_sentence']

frames = [common_df_final,just_zero, just_one]
result = pd.concat(frames)

In [ ]:
result['WI_positive_word'] = np.where(result['lable']== 1 , 1- result['frequent_negative_sentence']/ result['frequent_positive_sentence'], None)
result['WI_negative_word'] = np.where(result['lable']== 0 , 1- result['frequent_positive_sentence']/ result['frequent_negative_sentence'], None)
result['WI'] = result['WI_positive_word'].fillna(result['WI_negative_word'])

#result.index[result['lable'] == 0].tolist()

In [ ]:
df = pd.DataFrame(list(zip(text, y)), columns =['text', 'lable'])

In [ ]:
word_have_list= list(result.word)
word_have_set = set(word_have_list)
result_word_list = result['word'].tolist()
result_WI_list = result['WI'].tolist()

def word_WI_finder(x):
    df_words = set(x.split(' '))
    extract_words =  word_set.intersection(df_words)
    #index_word = result_word_list.index(extract_words)
    
    
    return (extract_words)


word_set = word_have_set

df['ddd_WI'] = df.text.apply(word_WI_finder)

In [ ]:
WI_ww = []
for x in df['ddd_WI'][:]:
    WI_w = []
    for y in x : 
        ind = result_word_list.index(y)
        WI_w.append(result_WI_list[ind])
    WI_ww.append(WI_w)
    
WI_ww_new = [[] if x==[0] else x for x in WI_ww]
WI_ww_neww = [[] if x==[0,0] else x for x in WI_ww_new]
Weight_sentence = [[float(j)/sum(i) for j in i] for i in WI_ww_neww[:]]



In [ ]:
sentence_ss = []
for i in range(len(df['ddd_WI'][:])):
    #print(df['ddd_WI'][x])
    vv = []
    for j in range(len(df['ddd_WI'][i])):
        #print(list(df['ddd_WI'][i])[j])
        if len(Weight_sentence[i]) > 1:
            v = []
            for num in range(d):
                vec = Weight_sentence[i][j] * float(model.get_word_vector(list(df['ddd_WI'][i])[j])[num])
                v.append(vec)
            #print(v)
            vv.append(v)
        #print(vv)
        else:
            vv.append(model.get_word_vector(list(df['ddd_WI'][i])[j]).astype(np.float))
            
    s = [sum(x) for x in zip(*vv)]
    sentence_ss.append(s)

sentence_ss[0]

data = pd.DataFrame(sentence_ss[0:] , columns = list(range(d)))


In [ ]:
r = [a for a in range(len(data))]
random.shuffle(r)
train_size = int(0.9*(len(r)))
test_size = int(0.1*(len(r)))
train_index = r[:train_size]
test_index = r[train_size:]
x_train = data.iloc[train_index]
x_test = data.iloc[test_index]

li_y_train = [yy[i] for i in train_index]
li_y_test = [yy[i] for i in test_index]
y_train = pd.DataFrame(li_y_train)
y_test = pd.DataFrame(li_y_test)
y_train = pd.DataFrame(li_y_train)
y_test = pd.DataFrame(li_y_test)

### xgboost

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(x_train , label = y_train)
dtest = xgb.DMatrix(x_test , label = y_test)
param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.1,  # the training step for each iteration
    #'silent': 1,  # logging mode - quiet
    'objective': 'binary:logistic',  
    'eval_metric': 'error',
    #'num_class': 2}  # the number of classes that exist in this datset
}

num_round = 500  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)
import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, best_preds , normalize=True)*100)

45.55555555555556


### CNN

In [ ]:
from keras import optimizers
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Embedding, Dropout
from keras.layers import GlobalMaxPool1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional,Conv2D
from keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
#https://stackoverflow.com/questions/62475807/cnn-on-tfidf-as-input
# create model
inp = Input(shape=(1,300))
conv2 = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(inp)
drop21 = Dropout(0.2)(conv2)

conv22 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(drop21)
drop22 = Dropout(0.2)(conv22)
conv23 = Conv1D(filters=32, kernel_size=5, activation='relu', padding='same')(drop22)
drop23 = Dropout(0.2)(conv23)
conv24 = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(drop23)
drop24 = Dropout(0.2)(conv24)

pool2 = Flatten()(drop24) # this is an option to pass from 3d to 2d
out = Dense(2, activation='softmax')(pool2) # the output dim must be equal to the num of class if u use softmax

model = Model(inp, out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(np.expand_dims(np.array(x_train), axis=1), np.array(y_train).flatten(), epochs=200)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 conv1d (Conv1D)             (None, 1, 128)            192128    
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 64)             41024     
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1, 32)             10272     
                                                                 
 dropout_2 (Dropout)         (None, 1, 32)             0     

In [ ]:
model.evaluate(np.expand_dims(np.array(x_test), axis=1), np.array(y_test).flatten(), batch_size=32, verbose=1)

3/3 [==============================] - 0s 8ms/step - loss: 2.5024 - accuracy: 0.8111


[2.50238299369812, 0.8111110925674438]

### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8333333333333334

### MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(7, 2), random_state=1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8333333333333334

### DecisionTree

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7111111111111111

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7333333333333333

### SVM

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8444444444444444

### Kmeans

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(x_train)
y_pred = kmeans.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.43333333333333335

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=3, min_samples=2).fit(x_train)
y_pred = DBSCAN(eps=3, min_samples=2).fit_predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.45555555555555555